In [106]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import os, sys
import time
from PIL import Image

In [108]:
automation_dir = "/mnt/analysis/e17023/Adam/GADGET2/"
ptype_change_rate = 20

while True:
    try:
        attribute_df = pd.read_csv(automation_dir + "AttributesLog.csv")
        param_df = pd.read_csv(automation_dir + "parameters.csv")

        # only consider complete simulations
        param_df = param_df[param_df['Status'] >= 4]

        combined_df = pd.merge(param_df, attribute_df, on="Sim")
        combined_df['QueueTime'] = combined_df['Sim'].apply(lambda x: int(x.split('-')[-1]))
        combined_df['PType'] = combined_df['Sim'].apply(lambda x: x.split('-')[0][1:])

        combined_df = combined_df.sort_values(by=['QueueTime'])
        
        param_images = []
        
        # plot queue time vs parameter for each parameter
        for param in param_df.columns:
            if param not in ['Sim', 'Status', 'N', 'P0', 'E0', 'P1', 'E1', 'Score', 'Xb', 'Yb', 'Seed', 'CD', 'CDH']:
                if combined_df[param].min() == combined_df[param].max():
                    continue
                plt.figure()
                plt.title(param)
                plt.xlabel('Time')
                plt.ylabel(param)
                plt.ylim(combined_df[param].min(), combined_df[param].max())
                plt.scatter(combined_df['QueueTime'], combined_df[param], s=1, c='k')
                if len(combined_df) > 30:
                    plt.plot(combined_df['QueueTime'], combined_df[param].rolling(30).mean(), c='r')
                plt.savefig(automation_dir + 'Output/images/' + param + '.png')
                plt.close()
                param_images.append(automation_dir + 'Output/images/' + param + '.png')
        
        attribute_images = {}
        # plot queue time vs attribute for each ptype and attribute
        for ptype in combined_df['PType'].unique():
            attribute_images[ptype] = []
            for attribute in attribute_df.columns:
                if attribute not in ['Sim', 'Energy']:
                    plt.figure()
                    plt.title(ptype + ' ' + attribute)
                    plt.xlabel('Time')
                    plt.ylabel(attribute)
                    #plt.ylim(combined_df[attribute].min(), combined_df[attribute].max())
                    plt.scatter(combined_df[combined_df['PType'] == ptype]['QueueTime'], combined_df[combined_df['PType'] == ptype][attribute], s=1, c='k')
                    if len(combined_df[combined_df['PType'] == ptype]) > 30:
                        plt.plot(combined_df[combined_df['PType'] == ptype]['QueueTime'], combined_df[combined_df['PType'] == ptype][attribute].rolling(30).mean(), c='r')
                    plt.axhline(attribute_df[attribute_df['Sim'] == ptype][attribute].mean(), c='b', ls='--')
                    plt.savefig(automation_dir + 'Output/images/' + ptype + '_' + attribute + '.png')
                    plt.close()
                    attribute_images[ptype].append(automation_dir + 'Output/images/' + ptype + '_' + attribute + '.png')
            
            plt.figure()
            plt.title(ptype + ' Score')
            plt.xlabel('Time')
            plt.ylabel('Score')
            plt.scatter(combined_df[combined_df['PType'] == ptype]['QueueTime'], combined_df[combined_df['PType'] == ptype]['Score'], c='k', s=1)
            if len(combined_df[combined_df['PType'] == ptype]) > 30:
                plt.plot(combined_df[combined_df['PType'] == ptype]['QueueTime'], combined_df[combined_df['PType'] == ptype]['Score'].rolling(30).min(), c='r')
            plt.axhline([0], c='b', ls='--')
            plt.savefig(automation_dir + 'Output/images/' + ptype + '_Score.png')
            plt.close()
            attribute_images[ptype].append(automation_dir + 'Output/images/' + ptype + '_Score.png')
        
        ptypes = list(attribute_images.keys())
        selected_ptype = ptypes[int(time.time() // ptype_change_rate % len(ptypes))]
        
        images = param_images + attribute_images[selected_ptype]
        
        # combine all images into one image in roughly a square shape
        images = [Image.open(x) for x in images]
        widths, heights = zip(*(i.size for i in images))
        
        total_width = int(np.ceil(np.sqrt(len(images))))
        total_height = int(np.ceil(np.sqrt(len(images))))
        
        new_im = Image.new('RGB', (total_width * max(widths), total_height * max(heights)))
        
        x_offset = 0
        y_offset = 0
        for im in images:
            new_im.paste(im, (x_offset, y_offset))
            x_offset += im.size[0]
            if x_offset >= total_width * max(widths):
                x_offset = 0
                y_offset += im.size[1]
        
        new_im.save(automation_dir + 'tuning_view.png')
        
    except pd.errors.EmptyDataError:
        time.sleep(1)
    except:
        plt.close()
        break